# Debugging autoreload

In [ ]:
%load_ext autoreload
%autoreload 2

# Load packages

In [ ]:
from pytorch_tabular.utils import load_covertype_dataset
from rich.pretty import pprint
from sklearn.model_selection import BaseCrossValidator, ParameterGrid, ParameterSampler
import torch
from scipy.stats import mannwhitneyu
import pickle
import shap
from sklearn.model_selection import RepeatedStratifiedKFold
from glob import glob
import ast
import matplotlib.pyplot as plt
import seaborn as sns
import copy
from sklearn.model_selection import train_test_split
import numpy as np
from pytorch_tabular.utils import make_mixed_dataset, print_metrics
from pytorch_tabular import available_models
from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig, GANDALFConfig, TabNetModelConfig, FTTransformerConfig, DANetConfig
from pytorch_tabular.config import DataConfig, OptimizerConfig, TrainerConfig
from pytorch_tabular.models.common.heads import LinearHeadConfig
from pytorch_tabular.tabular_model_tuner import TabularModelTuner
from torchmetrics.functional.regression import mean_absolute_error, pearson_corrcoef
from pytorch_tabular import MODEL_SWEEP_PRESETS
import pandas as pd
from pytorch_tabular import model_sweep
from src.pt.model_sweep import model_sweep_custom
import warnings
from src.utils.configs import read_parse_config
from src.utils.hash import dict_hash
import pathlib
from src.models.simage.tabular.widedeep.ft_transformer import WDFTTransformerModel
import os
from tqdm import tqdm
import optuna


# Load data

## Load SImAge

In [ ]:
path_simage = f"D:/YandexDisk/Work/pydnameth/datasets/GPL21145/GSEUNN/data/immuno/models/SImAge/best_fold_0002.ckpt"
feats_simage = pd.read_excel(f"D:/YandexDisk/Work/pydnameth/datasets/GPL21145/GSEUNN/data/immuno/models/SImAge/feats_con_top10.xlsx", index_col=0).index.values
model_simage = WDFTTransformerModel.load_from_checkpoint(checkpoint_path=path_simage)
model_simage.eval()
model_simage.freeze()
model_simage.to('cpu')

## Load immuno models

In [ ]:
%%capture

epi_data_type = 'no_harm'
selection_method = 'f_regression' # 'f_regression' 'spearman'
n_feats = 100
path_imm_models = f"D:/YandexDisk/Work/bbd/immunology/003_EpImAge/{epi_data_type}/{selection_method}_{n_feats}"

models_type = "models(GANDALF_616)_tst(5)_val(5)_CosineAnnealingWarmRestarts_yeo-johnson"

models = {}
models_paths = {}

for f in feats_simage:
    models[f] = {}
    models_paths[f] = []
    paths = glob(f"{path_imm_models}/{f}/pytorch_tabular/candidates/{models_type}/*/model.ckpt")
    for p in paths:
        head, tail = os.path.split(p)
        head = head.replace('\\', '/')
        models_paths[f].append(head)
        models[f][head] = TabularModel.load_model(f"{head}")

## Load data

In [ ]:
path_data = f"D:/YandexDisk/Work/pydnameth/datasets/GPL21145/GSEUNN/special/060_EpiSImAge"

ages_pc = {
    'PCHorvath1': 'PC-Horvath',
    'PCHorvath2': 'PC-SkinBloodAge',
    'PCHannum': 'PC-Hannum',
    'PCPhenoAge': 'PC-PhenoAge',
    'PCGrimAge': 'PC-GrimAge',
}
pace = 'DunedinPACE'
ages_calc = {
    'DNAmAge': 'Horvath',
    'DNAmAgeHannum': 'Hannum',
    'DNAmPhenoAge': 'PhenoAge',
    'DNAmAgeSkinBloodClock': 'SkinBloodAge',
    'DNAmGrimAgeBasedOnRealAge': 'GrimAge',
    'DNAmGrimAge2BasedOnRealAge': 'GrimAge2',
}
ages_all = list(ages_pc.values()) + list(ages_calc.values()) + [pace]

feats_epi_dict = {}
feats_epi_all = set()
for f in feats_simage:
    feats_epi_dict[f] = pd.read_excel(f"{path_imm_models}/{f}/feats_con.xlsx", index_col=0).index.values
    feats_epi_all.update(feats_epi_dict[f])
feats_epi_all = list(feats_epi_all)
print(len(feats_epi_all))

data_parts = ['GSEUNN', 'GSE87571', 'GSE40279', 'GSE179325', 'GSE217633', 'GSE118144', 'GSE42861', 'GSE106648', 'GSE67530', 'GSE77696']
dfs = []
for data_part in tqdm(data_parts):
    if data_part == 'GSEUNN':
        df_betas = pd.read_pickle(f"{path_data}/{data_part}/{epi_data_type}/betas.pkl")
        df_betas = df_betas.loc[:, feats_epi_all]
        df_pheno = pd.read_csv(f"{path_data}/{data_part}/{epi_data_type}/pheno.csv", index_col=0)
        df_pheno.rename(columns=ages_pc, inplace=True)
        df_horvath = pd.read_csv(glob(f"{path_data}/{data_part}/{epi_data_type}/DNAmAgeCalcProject_*_Results.csv")[0], index_col=0)
        for age_col, age_label in ages_calc.items():
            df_pheno.loc[df_pheno.index.values, age_label] = df_horvath.loc[df_pheno.index.values, age_col]
        df_pheno.set_index("index", inplace=True)
        cols_pheno = ['Age', 'Sex', 'Status', 'Region'] + ages_all
        df = pd.merge(df_pheno.loc[:, cols_pheno], df_betas, left_index=True, right_index=True)
    else:
        df_betas = pd.read_pickle(f"{path_data}/{data_part}/betas.pkl")
        df_betas = df_betas.loc[:, feats_epi_all]
        df_pheno = pd.read_csv(f"{path_data}/{data_part}/pheno.csv", index_col=0)
        df_pheno.rename(columns=ages_pc, inplace=True)
        df_horvath = pd.read_csv(glob(f"{path_data}/{data_part}/DNAmAgeCalcProject_*_Results.csv")[0], index_col=0)
        for age_col, age_label in ages_calc.items():
            df_pheno.loc[df_pheno.index.values, age_label] = df_horvath.loc[df_pheno.index.values, age_col]
        if data_part == 'GSE40279':
            df_pheno.set_index("gsm", inplace=True)
            df_pheno['Status'] = 'Control'
            cols_pheno = ['Age', 'Sex', 'Status'] + ages_all
            df = pd.merge(df_pheno.loc[:, cols_pheno], df_betas, left_index=True, right_index=True)
        elif data_part == 'GSE87571':
            df_pheno['Status'] = 'Control'
            cols_pheno = ['Age', 'Sex', 'Status'] + ages_all
            df = pd.merge(df_pheno.loc[:, cols_pheno], df_betas, left_index=True, right_index=True)
        elif data_part == 'GSE179325':
            cols_pheno = ['Age', 'Sex', 'Status'] + ages_all
            df = pd.merge(df_pheno.loc[:, cols_pheno], df_betas, left_index=True, right_index=True)
        elif data_part == 'GSE217633':
            df_pheno.rename(columns={'Years.with.HIV': 'Years with HIV'}, inplace=True)
            cols_pheno = ['Age', 'Sex', 'Status', 'Years with HIV'] + ages_all
            df = pd.merge(df_pheno.loc[:, cols_pheno], df_betas, left_index=True, right_index=True)
        elif data_part == 'GSE118144':
            cols_pheno = ['Age', 'Sex', 'Status'] + ages_all
            df = pd.merge(df_pheno.loc[:, cols_pheno], df_betas, left_index=True, right_index=True)
        elif data_part == 'GSE42861':
            cols_pheno = ['Age', 'Sex', 'Status'] + ages_all
            df = pd.merge(df_pheno.loc[:, cols_pheno], df_betas, left_index=True, right_index=True)
        elif data_part == 'GSE106648':
            cols_pheno = ['Age', 'Sex', 'Status'] + ages_all
            df = pd.merge(df_pheno.loc[:, cols_pheno], df_betas, left_index=True, right_index=True)
            df.drop('GSM2844233', inplace=True)
        elif data_part == 'GSE67530':
            cols_pheno = ['Age', 'Sex', 'Status'] + ages_all
            df = pd.merge(df_pheno.loc[:, cols_pheno], df_betas, left_index=True, right_index=True)
        elif data_part == 'GSE71955':
            cols_pheno = ['Age', 'Sex', 'Status'] + ages_all
            df = pd.merge(df_pheno.loc[:, cols_pheno], df_betas, left_index=True, right_index=True)
        elif data_part == 'GSE77696':
            df_pheno.set_index("Index", inplace=True)
            cols_pheno = ['Age', 'Sex', 'Status'] + ages_all
            df = pd.merge(df_pheno.loc[:, cols_pheno], df_betas, left_index=True, right_index=True)
    df['GSE'] = data_part
    dfs.append(df)

df = pd.concat(dfs, verify_integrity=True)

for age_type in list(ages_pc.values()) + list(ages_calc.values()):
    df[f'{age_type} acceleration'] = df[age_type] - df['Age']
    df[f'|{age_type} acceleration|'] = df[f'{age_type} acceleration'].abs()

ids_groups = {
    'GSEUNN\nControls': df.index[(df['GSE'] == 'GSEUNN') & (df['Status'] == 'Control')].values,
    'GSEUNN\nESRD': df.index[(df['GSE'] == 'GSEUNN') & (df['Status'] == 'ESRD')].values,
    'GSE87571': df.index[(df['GSE'] == 'GSE87571')].values,
    'GSE40279': df.index[(df['GSE'] == 'GSE40279')].values,
    'GSE179325\nControls': df.index[(df['GSE'] == 'GSE179325') & (df['Status'] == 'Control')].values,
    'GSE179325\nCOVID-19\nSevere': df.index[(df['GSE'] == 'GSE179325') & (df['Status'] == 'COVID-19 Severe')].values,
    'GSE179325\nCOVID-19\nMild': df.index[(df['GSE'] == 'GSE179325') & (df['Status'] == 'COVID-19 Mild')].values,
    'GSE217633\nControls': df.index[(df['GSE'] == 'GSE217633') & (df['Status'] == 'Control')].values,
    'GSE217633\nPre ART': df.index[(df['GSE'] == 'GSE217633') & (df['Status'] == 'Pre ART')].values,
    'GSE217633\nPost ART': df.index[(df['GSE'] == 'GSE217633') & (df['Status'] == 'Post ART')].values,
    'GSE118144\nControls': df.index[(df['GSE'] == 'GSE118144') & (df['Status'] == 'Control')].values,
    'GSE118144\nSLE': df.index[(df['GSE'] == 'GSE118144') & (df['Status'] == 'SLE')].values,
    'GSE42861\nControls': df.index[(df['GSE'] == 'GSE42861') & (df['Status'] == 'Control')].values,
    'GSE42861\nRheumatoid\nArthritis': df.index[(df['GSE'] == 'GSE42861') & (df['Status'] == 'Rheumatoid Arthritis')].values,
    'GSE106648\nControls': df.index[(df['GSE'] == 'GSE106648') & (df['Status'] == 'Control')].values,
    'GSE106648\nMultiple\nSclerosis': df.index[(df['GSE'] == 'GSE106648') & (df['Status'] == 'Multiple Sclerosis')].values,
    'GSE67530\nControls': df.index[(df['GSE'] == 'GSE67530') & (df['Status'] == 'Control')].values,
    'GSE67530\nICU': df.index[(df['GSE'] == 'GSE67530') & (df['Status'] == 'ICU')].values,
    'GSE67530\nICU ARDS': df.index[(df['GSE'] == 'GSE67530') & (df['Status'] == 'ICU ARDS')].values,
    'GSE77696\nControls': df.index[(df['GSE'] == 'GSE77696') & (df['Status'] == 'Control')].values,
    'GSE77696\nHIV': df.index[(df['GSE'] == 'GSE77696') & (df['Status'] == 'HIV')].values,
}
for group, ids in ids_groups.items():
    df.loc[ids, 'Group'] = group

mw_pairs = [
    ('GSEUNN\nControls', 'GSEUNN\nESRD'),
    ('GSEUNN\nControls', 'GSE87571'),
    ('GSEUNN\nControls', 'GSE40279'),
    ('GSE87571', 'GSE40279'),
    ('GSEUNN\nControls', 'GSE179325\nControls'),
    ('GSE179325\nControls', 'GSE179325\nCOVID-19\nMild'),
    ('GSE179325\nCOVID-19\nMild', 'GSE179325\nCOVID-19\nSevere'),
    ('GSE179325\nControls', 'GSE179325\nCOVID-19\nSevere'),
    ('GSEUNN\nControls', 'GSE217633\nControls'),
    ('GSE217633\nControls', 'GSE217633\nPre ART'),
    ('GSE217633\nPre ART', 'GSE217633\nPost ART'),
    ('GSE217633\nControls', 'GSE217633\nPost ART'),
    ('GSEUNN\nControls', 'GSE118144\nControls'),
    ('GSE118144\nControls', 'GSE118144\nSLE'),
    ('GSEUNN\nControls', 'GSE42861\nControls'),
    ('GSE42861\nControls', 'GSE42861\nRheumatoid\nArthritis'),
    ('GSEUNN\nControls', 'GSE106648\nControls'),
    ('GSE106648\nControls', 'GSE106648\nMultiple\nSclerosis'),
    ('GSEUNN\nControls', 'GSE67530\nControls'),
    ('GSE67530\nControls', 'GSE67530\nICU'),
    ('GSE67530\nControls', 'GSE67530\nICU ARDS'),
    ('GSE67530\nICU', 'GSE67530\nICU ARDS'),
    ('GSEUNN\nControls', 'GSE77696\nControls'),
    ('GSE77696\nControls', 'GSE77696\nHIV'),
]

first_columns = [
    "MAE\nGSEUNN\nControls",
    "<AA>\nGSEUNN\nControls",
    "MAE\nGSE87571",
    "<AA>\nGSE87571",
    "MAE\nGSE40279",
    "<AA>\nGSE40279",
    "MAE\nGSE179325\nControls",
    "<AA>\nGSE179325\nControls",
    "MAE\nGSE217633\nControls",
    "<AA>\nGSE217633\nControls",
    "MAE\nGSE118144\nControls",
    "<AA>\nGSE118144\nControls",
    "MAE\nGSE42861\nControls",
    "<AA>\nGSE42861\nControls",
    "MAE\nGSE106648\nControls",
    "<AA>\nGSE106648\nControls",
    "MAE\nGSE67530\nControls",
    "<AA>\nGSE67530\nControls",
    "MAE\nGSE77696\nControls",
    "<AA>\nGSE77696\nControls",
] + [f"Mann-Whitney\n{pair[0]}\nVS\n{pair[1]}" for pair in mw_pairs]


# Check models' combinations

## Define objective function for optuna

In [ ]:
results_combos = []

def objective(trial: optuna.Trial):
    params = {}
    res_dict = {}
    for f in feats_simage:
        params[f] = trial.suggest_categorical(f, models_paths[f])
        res_dict[f] = params[f]
    
    for f in feats_simage:
        df[f"{f}_log"] = models[f][params[f]].predict(df.loc[:, feats_epi_dict[f]])
        df[f] = np.exp(df[f"{f}_log"])
    
    df['EpiSImAge'] = model_simage(torch.from_numpy(df.loc[:, feats_simage].values)).cpu().detach().numpy().ravel()
    df['EpiSImAge Acceleration'] = df['EpiSImAge'] - df['Age']
    df['|EpiSImAge Acceleration|'] = df['EpiSImAge Acceleration'].abs()
    
    for group, ids in ids_groups.items():
        pred = torch.from_numpy(np.float32(df.loc[ids, 'EpiSImAge'].values))
        real = torch.from_numpy(np.float32(df.loc[ids, 'Age'].values))
        res_dict[f"MAE\n{group}"] = mean_absolute_error(pred, real).item()
        res_dict[f"Pearson\n{group}"] = pearson_corrcoef(pred, real).item()
        res_dict[f"<AA>\n{group}"] = np.mean(df.loc[ids, 'EpiSImAge Acceleration'].values)
    
    for pair in mw_pairs:
        stat, pval = mannwhitneyu(
            df.loc[ids_groups[pair[0]], 'EpiSImAge Acceleration'].values,
            df.loc[ids_groups[pair[1]], 'EpiSImAge Acceleration'].values,
            alternative='two-sided'
        )
        res_dict[f"Mann-Whitney\n{pair[0]}\nVS\n{pair[1]}"] = pval
    
    results_combos.append(res_dict)
    
    optimized_metric_to_min_1 = res_dict["MAE\nGSE87571"] + res_dict["MAE\nGSE40279"]
    mw_pair_1 = ('GSEUNN\nControls', 'GSE87571')
    optimized_metric_to_max_1 = -np.log10(res_dict[f"Mann-Whitney\n{mw_pair_1[0]}\nVS\n{mw_pair_1[1]}"])
    mw_pair_2 = ('GSEUNN\nControls', 'GSE40279')
    optimized_metric_to_max_2 = -np.log10(res_dict[f"Mann-Whitney\n{mw_pair_2[0]}\nVS\n{mw_pair_2[1]}"])
    
    return optimized_metric_to_min_1, optimized_metric_to_max_1, optimized_metric_to_max_2

## Run optimization

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

results_combos = []

n_trials = 5000

study = optuna.create_study(study_name='optimization', directions=["minimize", "minimize", "minimize"])
study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

In [ ]:
df_combos = pd.DataFrame(results_combos)
df_combos = df_combos[first_columns + [col for col in df_combos.columns if col not in first_columns]]
df_combos.to_excel(f"{path_imm_models}/EpiSImAge_combos.xlsx")

In [ ]:
optuna.importance.get_param_importances(study, target=lambda t: t.values[0])